# Data Cleaning

This notebook performs exploratory data inspection and cleaning of the synthetic campaign dataset using SQL, and Python. It addresses null values, inconsistent text formats, outliers, and prepares the dataset for analysis.

In [1]:
import pandas as pd


# Load your data
df = pd.read_csv('data/campaign_data.csv')
type(df)

pandas.core.frame.DataFrame

In [2]:
from IPython.display import display
display(df.head())

,customer_id,age,gender,income,region,tenure_years,received_campaign,spend_last_month,channel,product_category,responded,spend_this_month,is_high_value,days_since_last_purchase,credit_score,customer_segment
0,CUST00001,56,female,111974,North,4.5,0,616.04,email,Loans,1,771.72,0,156,664,1
1,CUST00002,69,man,42291,South,5.7,0,171.21,email,Loans,0,171.21,0,185,732,2
2,CUST00003,46,female,70585,North,5.7,0,280.33,sms,Loans,0,280.33,0,41,715,0
3,CUST00004,32,Male,78626,East,9.3,0,143.79,sms,NaN,0,143.79,0,201,725,0
4,CUST00005,60,Female,64871,South,0.0,1,67.43,email,NaN,1,221.13,0,50,719,3


In [3]:
import sqlite3
# Connect to an in-memory SQLite DB
conn = sqlite3.connect(':memory:')

# Write the dataframe to SQL table
df.to_sql('campaign_data', conn, index=False, if_exists='replace')

# Define your SQL queries for inconsistencies:

queries = {
    'Gender case inconsistency': """
        SELECT gender, COUNT(*) as count
        FROM campaign_data
        GROUP BY gender
        ORDER BY count DESC;
    """,

    'Channel case inconsistency': """
        SELECT channel, COUNT(*) as count
        FROM campaign_data
        GROUP BY channel
        ORDER BY count DESC;
    """,

    'Region unknown or invalid': """
        SELECT region, COUNT(*) as count
        FROM campaign_data
        WHERE region = 'Unknown' OR region IS NULL OR region NOT IN ('North', 'South', 'East', 'West')
        GROUP BY region;
    """,

    'Region case inconsistency': """
        SELECT region, COUNT(*) as count
        FROM campaign_data
        GROUP BY region
        ORDER BY region DESC;
    """,

    'Null or missing values count per column': """
        SELECT
            SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS age_nulls,
            SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS gender_nulls,
            SUM(CASE WHEN income IS NULL THEN 1 ELSE 0 END) AS income_nulls,
            SUM(CASE WHEN region IS NULL THEN 1 ELSE 0 END) AS region_nulls,
            SUM(CASE WHEN tenure_years IS NULL THEN 1 ELSE 0 END) AS tenure_nulls
        FROM campaign_data;
    """,

    'Invalid age values (less than 18 or greater than 100)': """
        SELECT age, COUNT(*) as count
        FROM campaign_data
        WHERE age < 18 OR age > 100
        GROUP BY age;
    """,

    'Product_category unknown or invalid': """
        SELECT product_category, COUNT(*) as count
        FROM campaign_data
        GROUP BY product_category;
    """
}

# Run and display query results
for name, query in queries.items():
    print(f"\n--- {name} ---")
    result = pd.read_sql_query(query, conn)
    print(result)


--- Gender case inconsistency ---
   gender  count
0    Male   1348
1  female   1209
2  Female   1206
3     man   1108
4   Other    129

--- Channel case inconsistency ---
            channel  count
0  App Notification    733
1             email    724
2             phone    721
3             Phone    721
4               sms    718
5               SMS    711
6             Email    672

--- Region unknown or invalid ---
    region  count
0  Unknown    207

--- Region case inconsistency ---
    region  count
0     West   1228
1  Unknown    207
2    South   1183
3    North   1143
4     East   1239

--- Null or missing values count per column ---
   age_nulls  gender_nulls  income_nulls  region_nulls  tenure_nulls
0          0             0             0             0             0

--- Invalid age values (less than 18 or greater than 100) ---
Empty DataFrame
Columns: [age, count]
Index: []

--- Product_category unknown or invalid ---
  product_category  count
0             None   1006
1 

It is clear that there is inconsistency in Gender, Channel, and Region columns.

## 1- Fix gender column using SQL

In [4]:
#!pip install -q pandasql

Import it and write the SQL query:

In [5]:
import pandasql as ps

# SQL query to create a new column with cleaned gender values
query = """
SELECT *,
  CASE
    WHEN LOWER(gender) IN ('male', 'man') THEN 'Male'
    WHEN LOWER(gender) = 'female' THEN 'Female'
    WHEN LOWER(gender) = 'other' THEN 'Other'
    ELSE 'Unknown'
  END AS gender_cleaned
FROM df
"""

df_cleaned = ps.sqldf(query)

Preview cleaned data:

In [6]:
df_cleaned[['gender', 'gender_cleaned']].value_counts().reset_index(name='count')

,gender,gender_cleaned,count
0,Male,Male,1348
1,female,Female,1209
2,Female,Female,1206
3,man,Male,1108
4,Other,Other,129


In [7]:

df['gender_cleaned'] = df_cleaned['gender_cleaned']

display(df.head())

,customer_id,age,gender,income,region,tenure_years,received_campaign,spend_last_month,channel,product_category,responded,spend_this_month,is_high_value,days_since_last_purchase,credit_score,customer_segment,gender_cleaned
0,CUST00001,56,female,111974,North,4.5,0,616.04,email,Loans,1,771.72,0,156,664,1,Female
1,CUST00002,69,man,42291,South,5.7,0,171.21,email,Loans,0,171.21,0,185,732,2,Male
2,CUST00003,46,female,70585,North,5.7,0,280.33,sms,Loans,0,280.33,0,41,715,0,Female
3,CUST00004,32,Male,78626,East,9.3,0,143.79,sms,NaN,0,143.79,0,201,725,0,Male
4,CUST00005,60,Female,64871,South,0.0,1,67.43,email,NaN,1,221.13,0,50,719,3,Female


## 2- Fix channel columns using Python

In [8]:
# Normalize case and whitespace
df['channel'] = df['channel'].str.strip().str.lower()

# Mapping to consistent values
channel_map = {
    'email': 'Email',
    'sms': 'SMS',
    'phone': 'Phone',
    'app notification': 'App Notification'
}

# Apply mapping
df['channel_cleaned'] = df['channel'].replace(channel_map)

display(df.head())

,customer_id,age,gender,income,region,tenure_years,received_campaign,spend_last_month,channel,product_category,responded,spend_this_month,is_high_value,days_since_last_purchase,credit_score,customer_segment,gender_cleaned,channel_cleaned
0,CUST00001,56,female,111974,North,4.5,0,616.04,email,Loans,1,771.72,0,156,664,1,Female,Email
1,CUST00002,69,man,42291,South,5.7,0,171.21,email,Loans,0,171.21,0,185,732,2,Male,Email
2,CUST00003,46,female,70585,North,5.7,0,280.33,sms,Loans,0,280.33,0,41,715,0,Female,SMS
3,CUST00004,32,Male,78626,East,9.3,0,143.79,sms,NaN,0,143.79,0,201,725,0,Male,SMS
4,CUST00005,60,Female,64871,South,0.0,1,67.43,email,NaN,1,221.13,0,50,719,3,Female,Email


## 3. Replace "Unknown" in region with mode

For simplicity, replacing "Unknown" in the region column with the mode sounds a clean and reasonable approach when there's no business logic to determine the true region.

In [9]:
# First normalize the original 'region' column
df['region_normalized'] = df['region'].str.strip().str.title()

# Then compute the mode (excluding 'Unknown')
region_mode = df.loc[df['region_normalized'] != 'Unknown', 'region_normalized'].mode()[0]
region_mode

'East'

In [10]:
# Create cleaned column with 'Unknown' replaced by the mode
df['region_cleaned'] = df['region_normalized'].replace('Unknown', region_mode)

In [11]:
df['region_cleaned'] = df['region'].replace(channel_map)
df[['region', 'region_cleaned']].head()

,region,region_cleaned
0,North,North
1,South,South
2,North,North
3,East,East
4,South,South


In [12]:
df[['region_cleaned']]

,region_cleaned
0,North
1,South
2,North
3,East
4,South
...,...
4995,Unknown
4996,East
4997,East
4998,South


In [13]:
print(df['region_cleaned'].value_counts())

region_cleaned
East       1239
West       1228
South      1183
North      1143
Unknown     207
Name: count, dtype: int64


Since I still see Unknown in region_cleaned, it means some Unknown values may have different casings or extra spaces, or the replacement didn't catch all variants.

Try a more thorough cleaning approach:

In [14]:
import numpy as np
# Normalize 'region' by stripping spaces and making consistent casing
df['region_normalized'] = df['region'].str.strip().str.title()

# Replace all variations of 'Unknown' (case-insensitive)
df['region_normalized'] = df['region_normalized'].replace(
    to_replace=r'Unknown',
    value=np.nan,
    regex=True
)

# Calculate mode excluding NaN values
region_mode = df['region_normalized'].mode()[0]

# Fill NaN with mode
df['region_cleaned'] = df['region_normalized'].fillna(region_mode)

# Check counts again
print(df['region_cleaned'].value_counts())

region_cleaned
East     1446
West     1228
South    1183
North    1143
Name: count, dtype: int64


In [15]:
display(df.head())

,customer_id,age,gender,income,region,tenure_years,received_campaign,spend_last_month,channel,product_category,responded,spend_this_month,is_high_value,days_since_last_purchase,credit_score,customer_segment,gender_cleaned,channel_cleaned,region_normalized,region_cleaned
0,CUST00001,56,female,111974,North,4.5,0,616.04,email,Loans,1,771.72,0,156,664,1,Female,Email,North,North
1,CUST00002,69,man,42291,South,5.7,0,171.21,email,Loans,0,171.21,0,185,732,2,Male,Email,South,South
2,CUST00003,46,female,70585,North,5.7,0,280.33,sms,Loans,0,280.33,0,41,715,0,Female,SMS,North,North
3,CUST00004,32,Male,78626,East,9.3,0,143.79,sms,NaN,0,143.79,0,201,725,0,Male,SMS,East,East
4,CUST00005,60,Female,64871,South,0.0,1,67.43,email,NaN,1,221.13,0,50,719,3,Female,Email,South,South


## 4. Replace Null values in product_category with mode in Python



In [16]:
'''
# Step 1: Get the mode (most frequent value)
mode_value = df['product_category'].mode()[0]
print(f"Mode of 'product_category': {mode_value}")


# Step 2: Fill NaN values with the mode
#df['product_category'].fillna(mode_value, inplace=True)
df.fillna({'product_category': mode_value}, inplace=True)
df.head()
'''

'\n# Step 1: Get the mode (most frequent value)\nmode_value = df[\'product_category\'].mode()[0]\nprint(f"Mode of \'product_category\': {mode_value}")\n\n\n# Step 2: Fill NaN values with the mode\n#df[\'product_category\'].fillna(mode_value, inplace=True)\ndf.fillna({\'product_category\': mode_value}, inplace=True)\ndf.head()\n'

## 4.Replace Null values in product_category with mode in SQL

In [17]:
import pandasql as ps

# Step 1: Get the mode value
mode_query = """
SELECT product_category
FROM df
WHERE product_category IS NOT NULL
GROUP BY product_category
ORDER BY COUNT(*) DESC
LIMIT 1
"""
mode_result = ps.sqldf(mode_query)
mode_value = mode_result['product_category'][0]

# Step 2: Fill NaNs in the DataFrame
df['product_category'] = df['product_category'].fillna(mode_value)
df.head()

,customer_id,age,gender,income,region,tenure_years,received_campaign,spend_last_month,channel,product_category,responded,spend_this_month,is_high_value,days_since_last_purchase,credit_score,customer_segment,gender_cleaned,channel_cleaned,region_normalized,region_cleaned
0,CUST00001,56,female,111974,North,4.5,0,616.04,email,Loans,1,771.72,0,156,664,1,Female,Email,North,North
1,CUST00002,69,man,42291,South,5.7,0,171.21,email,Loans,0,171.21,0,185,732,2,Male,Email,South,South
2,CUST00003,46,female,70585,North,5.7,0,280.33,sms,Loans,0,280.33,0,41,715,0,Female,SMS,North,North
3,CUST00004,32,Male,78626,East,9.3,0,143.79,sms,Loans,0,143.79,0,201,725,0,Male,SMS,East,East
4,CUST00005,60,Female,64871,South,0.0,1,67.43,email,Loans,1,221.13,0,50,719,3,Female,Email,South,South


## Double check inconsistencies

In [18]:
# Write the dataframe to SQL table
df.to_sql('campaign_data', conn, index=False, if_exists='replace')

queries = {
    'Gender case inconsistency': """
        SELECT gender_cleaned, COUNT(*) as count
        FROM campaign_data
        GROUP BY gender_cleaned
        ORDER BY count DESC;
    """,

    'Channel case inconsistency': """
        SELECT channel_cleaned, COUNT(*) as count
        FROM campaign_data
        GROUP BY channel_cleaned
        ORDER BY count DESC;
    """,

    'Region unknown or invalid': """
        SELECT region, COUNT(*) as count
        FROM campaign_data
        WHERE region = 'Unknown' OR region IS NULL OR region NOT IN ('North', 'South', 'East', 'West')
        GROUP BY region;
    """,

    'Region case inconsistency': """
        SELECT region_cleaned, COUNT(*) as count
        FROM campaign_data
        GROUP BY region_cleaned
        ORDER BY region DESC;
    """,

    'Null or missing values count per column': """
        SELECT
            SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS age_nulls,
            SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS gender_nulls,
            SUM(CASE WHEN income IS NULL THEN 1 ELSE 0 END) AS income_nulls,
            SUM(CASE WHEN region IS NULL THEN 1 ELSE 0 END) AS region_nulls,
            SUM(CASE WHEN tenure_years IS NULL THEN 1 ELSE 0 END) AS tenure_nulls
        FROM campaign_data;
    """,

    'Invalid age values (less than 18 or greater than 100)': """
        SELECT age, COUNT(*) as count
        FROM campaign_data
        WHERE age < 18 OR age > 100
        GROUP BY age;
    """,
    'Product_category unknown or invalid': """
        SELECT product_category, COUNT(*) as count
        FROM campaign_data
        GROUP BY product_category;
    """
}

# Run and display query results
for name, query in queries.items():
    print(f"\n--- {name} ---")
    result = pd.read_sql_query(query, conn)
    print(result)


--- Gender case inconsistency ---
  gender_cleaned  count
0           Male   2456
1         Female   2415
2          Other    129

--- Channel case inconsistency ---
    channel_cleaned  count
0             Phone   1442
1               SMS   1429
2             Email   1396
3  App Notification    733

--- Region unknown or invalid ---
    region  count
0  Unknown    207

--- Region case inconsistency ---
  region_cleaned  count
0           West   1228
1          South   1183
2          North   1143
3           East   1446

--- Null or missing values count per column ---
   age_nulls  gender_nulls  income_nulls  region_nulls  tenure_nulls
0          0             0             0             0             0

--- Invalid age values (less than 18 or greater than 100) ---
Empty DataFrame
Columns: [age, count]
Index: []

--- Product_category unknown or invalid ---
  product_category  count
0      Credit Card    997
1        Insurance    967
2            Loans   2036
3          Savings   1000

## Drop the original uncleaned columns

In [19]:
df = df.drop(columns=['region', 'region_normalized', 'gender', 'channel'])

## Rename the new columns

In [20]:
df = df.rename(columns={
    'region_cleaned': 'region',
    'gender_cleaned': 'gender',
    'channel_cleaned': 'channel'
})

# Define the desired column order
desired_order = [
    'customer_id', 'age', 'gender', 'income', 'region', 'tenure_years',
    'received_campaign', 'spend_last_month', 'channel', 'product_category',
    'responded', 'spend_this_month', 'is_high_value', 'days_since_last_purchase',
    'credit_score', 'customer_segment'
]

# Reorder the DataFrame columns
df = df[desired_order]

display(df.head())

,customer_id,age,gender,income,region,tenure_years,received_campaign,spend_last_month,channel,product_category,responded,spend_this_month,is_high_value,days_since_last_purchase,credit_score,customer_segment
0,CUST00001,56,Female,111974,North,4.5,0,616.04,Email,Loans,1,771.72,0,156,664,1
1,CUST00002,69,Male,42291,South,5.7,0,171.21,Email,Loans,0,171.21,0,185,732,2
2,CUST00003,46,Female,70585,North,5.7,0,280.33,SMS,Loans,0,280.33,0,41,715,0
3,CUST00004,32,Male,78626,East,9.3,0,143.79,SMS,Loans,0,143.79,0,201,725,0
4,CUST00005,60,Female,64871,South,0.0,1,67.43,Email,Loans,1,221.13,0,50,719,3


## Save Cleaned Data

In [21]:
df.to_csv('/content/sample_data/campaign_data_cleaned.csv', index=False)